# Retail Pricing Optimization Model
This notebook optimizes weekly pricing using a discrete price ladder and a linear demand model to maximize revenue over a 13-week horizon (excluding week 164).

### Import Required Libraries

In [1]:
%pip install gurobipy 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

### Define Weeks and Price Ladder

In [3]:
weeks = list(range(157, 170))
weeks.remove(164)  # Exclude week 164
p_ladder = [1.00, 0.95, 0.85, 0.75, 0.60, 0.50]  # Price ladder

### Map Weeks to Seasons

In [4]:
season = {}
for w in weeks:
    season[w] = int(np.ceil((w % 52) / 4))
print(season)

{157: 1, 158: 1, 159: 1, 160: 1, 161: 2, 162: 2, 163: 2, 165: 3, 166: 3, 167: 3, 168: 3, 169: 4}


### Define Demand Model Coefficients

In [5]:
intercept = 1.978242858
p_coeff = -2.809634145
p1_coeff = 0.963410728
p2_coeff = 0.759639170

season_coeff = {
    1: 0,
    2: -0.562046910, 3: 0.087545274, 4: -0.402637480,
    5: -0.027326010, 6: 0.004349885, 7: -0.036102297,
    8: -0.069280527, 9: 0.160276197, 10: 1.104208897,
    11: 1.122711287, 12: 1.176802194, 13: 0.947945548
}

### Build Optimization Model

In [6]:
mod = gp.Model("price_model_with_ladder")

# Decision variables
p = mod.addVars(weeks)
x = mod.addVars(weeks, p_ladder, vtype=GRB.BINARY)

# Each week selects one price from the ladder
mod.addConstrs((gp.quicksum(x[w, k] for k in p_ladder) == 1) for w in weeks)

# Price is equal to the selected ladder value
mod.addConstrs((p[w] == gp.quicksum(k * x[w, k] for k in p_ladder)) for w in weeks)

# Safe defaults for prior prices
default_p1 = 1.00
default_p2 = 1.00

# Objective function: maximize revenue
mod.setObjective(
    gp.quicksum(
        p[w] * (
            intercept +
            p_coeff * p[w] +
            p1_coeff * (p[w-1] if (w-1) in weeks else default_p1) +
            p2_coeff * (p[w-2] if (w-2) in weeks else default_p2) +
            season_coeff[season[w]]
        )
        for w in weeks
    ),
    GRB.MAXIMIZE
)

mod.optimize()

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.5.0 24F5053j)

CPU model: Apple M4 Max
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 24 rows, 84 columns and 156 nonzeros
Model fingerprint: 0x35c141f4
Model has 31 quadratic objective terms
Variable types: 12 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 8.0544624
Presolve removed 0 rows and 12 columns
Presolve time: 0.01s
Presolved: 24 rows, 72 columns, 132 nonzeros
Presolved model has 31 quadratic objective terms
Variable types: 12 continuous, 60 integer (60 binary)

Root relaxation: objective 1.052862e+01, 37 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    C

### View Optimized Prices

In [7]:
df = pd.DataFrame(index=weeks, columns=['price'])
for w in weeks:
    df.loc[w, 'price'] = p[w].X

df

,price
157,0.95
158,0.95
159,0.85
160,0.85
161,0.75
162,0.6
163,0.6
165,0.85
166,0.85
167,0.85


### Plot Results

In [8]:
fig = px.line(df, x=df.index, y='price', markers=True)
fig.update_layout(title='Optimized Weekly Prices', xaxis_title='Week', yaxis_title='Price', plot_bgcolor='white')
fig.show()